In [16]:
import chromadb
client = chromadb.PersistentClient('data/chroma-data')
#client = chromadb.HttpClient(host='localhost', port=8000)

In [17]:
texts_collection = client.create_collection(name='eidc_texts')
sentences_collection = client.create_collection(name='eidc_sentences')

In [18]:
from util import load_eidc_data
df = load_eidc_data.load_as_df('data/catalogue_metadata.json')
df = df.drop_duplicates(subset=['title'])

In [40]:
import nltk
eidc_sentences = []
eidc_metadata = []
for i, row in df.iterrows():
    document_sentences = nltk.sent_tokenize(row['description'])
    eidc_sentences.extend(document_sentences)
    eidc_metadata.extend([{'title': row['title']} for x in range(len(document_sentences))])
eidc_sentence_ids = [f'sent_{i:05d}' for i in range(len(eidc_sentences))]
# Individual sentence can be embedded and added to the db but are currently omitted 

11607:11607


In [34]:
eidc_docs = df['description'].to_list()
eidc_metadata = [{'title': title} for title in df['title'].to_list()]
eidc_ids = [f'doc_{i:04d}' for i in range(len(eidc_docs))]

['doc_0000',
 'doc_0001',
 'doc_0002',
 'doc_0003',
 'doc_0004',
 'doc_0005',
 'doc_0006',
 'doc_0007',
 'doc_0008',
 'doc_0009',
 'doc_0010',
 'doc_0011',
 'doc_0012',
 'doc_0013',
 'doc_0014',
 'doc_0015',
 'doc_0016',
 'doc_0017',
 'doc_0018',
 'doc_0019',
 'doc_0020',
 'doc_0021',
 'doc_0022',
 'doc_0023',
 'doc_0024',
 'doc_0025',
 'doc_0026',
 'doc_0027',
 'doc_0028',
 'doc_0029',
 'doc_0030',
 'doc_0031',
 'doc_0032',
 'doc_0033',
 'doc_0034',
 'doc_0035',
 'doc_0036',
 'doc_0037',
 'doc_0038',
 'doc_0039',
 'doc_0040',
 'doc_0041',
 'doc_0042',
 'doc_0043',
 'doc_0044',
 'doc_0045',
 'doc_0046',
 'doc_0047',
 'doc_0048',
 'doc_0049',
 'doc_0050',
 'doc_0051',
 'doc_0052',
 'doc_0053',
 'doc_0054',
 'doc_0055',
 'doc_0056',
 'doc_0057',
 'doc_0058',
 'doc_0059',
 'doc_0060',
 'doc_0061',
 'doc_0062',
 'doc_0063',
 'doc_0064',
 'doc_0065',
 'doc_0066',
 'doc_0067',
 'doc_0068',
 'doc_0069',
 'doc_0070',
 'doc_0071',
 'doc_0072',
 'doc_0073',
 'doc_0074',
 'doc_0075',
 'doc_0076',

In [35]:
from chromadb.utils import embedding_functions
model_name = 'multi-qa-mpnet-base-dot-v1'
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)
texts_collection = client.create_collection(name='eidc_texts_new', embedding_function=sentence_transformer_ef, metadata={"hnsw:space": "cosine"})
texts_collection.add(
    documents=eidc_docs,
    ids=eidc_ids,
    metadatas=eidc_metadata
)

/home/mpc/github/discoverability/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
You try to use a model that was created with version 3.0.0.dev0, however, your version is 3.0.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



/home/mpc/github/discoverability/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
queries = ['Where is the wettest soil in the UK?',
             'Where is water quality worst in the UK?',
             'Where are bird populations declining in the UK?',
             'Where in the UK are bumblebees most at risk from neonicotinoids?',
             'Which county in the UK has the most rivers?']

In [38]:
results = texts_collection.query(
    query_texts = queries,
    n_results = 5,
    include = ['metadatas']
)
results

{'ids': [['doc_1679', 'doc_1665', 'doc_0774', 'doc_1571', 'doc_1149'],
  ['doc_0290', 'doc_1641', 'doc_0165', 'doc_0595', 'doc_1665'],
  ['doc_1598', 'doc_0644', 'doc_1389', 'doc_1560', 'doc_0740'],
  ['doc_1755', 'doc_0341', 'doc_1436', 'doc_1656', 'doc_1754'],
  ['doc_1464', 'doc_0165', 'doc_1641', 'doc_1081', 'doc_1666']],
 'distances': None,
 'metadatas': [[{'title': 'Grid-to-Grid model estimates of soil moisture for Great Britain and Northern Ireland driven by observed data (1980 to 2011)'},
   {'title': 'Grid-to-Grid model estimates of soil moisture for Great Britain and Northern Ireland driven by UK Climate Projections 2018 (UKCP18) Regional (12km) data (1980 to 2080)'},
   {'title': 'Relative surface soil moisture for the Thames Valley, United Kingdom, October 2015 to September 2021'},
   {'title': 'Model estimates of topsoil moisture [Countryside Survey]'},
   {'title': 'Coastal Biodiversity and Ecosystem Service Sustainability (CBESS) soil moisture content from three soil dep

In [39]:
from tabulate import tabulate
md = tabulate(results['metadatas'])
print(md)

----------------------------------------------------------------------------------------------------------------------------------------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ---------------------------------------------------------------------------------------------------------------------------------------------  -------------------------------------------------------------------------------------------------------------------------------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'title': 'Grid-to-Grid model estimates of soil moisture for Great Britain and Northern Ireland driven by observed data (1980 to 2011)'}  {'title': 'Grid-to-Grid model estimates of soil moisture for Great Britain and Northern I

In [42]:
client.list_collections()

Settings(environment='', chroma_api_impl='chromadb.api.segment.SegmentAPI', chroma_server_nofile=None, chroma_server_thread_pool_size=40, tenant_id='default', topic_namespace='default', chroma_server_host=None, chroma_server_headers=None, chroma_server_http_port=None, chroma_server_ssl_enabled=False, chroma_server_ssl_verify=None, chroma_server_api_default_path='/api/v1', chroma_server_cors_allow_origins=[], is_persistent=True, persist_directory='data/chroma-data', chroma_memory_limit_bytes=0, chroma_segment_cache_policy=None, allow_reset=False, chroma_auth_token_transport_header=None, chroma_client_auth_provider=None, chroma_client_auth_credentials=None, chroma_server_auth_ignore_paths={'/api/v1': ['GET'], '/api/v1/heartbeat': ['GET'], '/api/v1/version': ['GET']}, chroma_overwrite_singleton_tenant_database_access_from_auth=False, chroma_server_authn_provider=None, chroma_server_authn_credentials=None, chroma_server_authn_credentials_file=None, chroma_server_authz_provider=None, chroma